In [1]:
import requests
import json
import pandas as pd
import os
import datetime as dt 
from pprint import pprint
%load_ext dotenv
%dotenv secrets.env

Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 8


In [61]:
stations_df = pd.read_csv('data_from_web/weather_stations.csv')
stations_df.drop(columns=['NCDCID', 'WBAN', 'COUNTY', 'ELEV', 'UTC', 'STNTYPE'], inplace=True)
stations_df.head()

,NAME,COUNTRY,ST,LAT,LON,GHCND
0,EGBERT1W,CANADA,ON,44.2326,-79.7810,CAW00064757
1,ALEKNAGIK1NNE,UNITEDSTATES,AK,59.2840,-158.6145,NaN
2,BETHEL87WNW,UNITEDSTATES,AK,61.3465,-164.0769,USW00026656
3,CORDOVA14ESE,UNITEDSTATES,AK,60.4731,-145.3542,USW00096405
4,DEADHORSE3S,UNITEDSTATES,AK,70.1618,-148.4644,USW00026565


In [65]:
stadium_df = pd.read_csv('data_from_web/another_stadiums.csv', delimiter=';')
stadium_df = stadium_df[['TEAM', 'NAME', 'Geo Point', 'ROOF_TYPE']]
stadium_df = stadium_df.append(pd.DataFrame({'TEAM': 'LOSANGELESCHARGERS/LOSANGELESRAMS', 'NAME':'Los Angeles Memorial Coliseum', 'Geo Point': '34.051,-118.1716', 'ROOF_TYPE':'Open'}, index=[31]))
for index1,row1 in stadium_df.iterrows():
    latDif = 10000
    longDif = 10000
    latitude, longitude = row1['Geo Point'].split(',')
    latitude = float(latitude)
    longitude= float(longitude)
    for index2, row2 in stations_df.iterrows():
        if abs(latitude-(row2['LAT'])) < latDif and abs(longitude-row2['LON'])<longDif:
            latDif = abs(latitude-row2['LAT'])
            longDif = abs(longitude-row2['LON'])
            stadium_df.loc[index1 , 'Station'] = row2['GHCND']
team = []
for index, row in stadium_df.iterrows():
    team.append(row['TEAM'].upper().replace(' ',''))
stadium_df['TEAM'] = team
stadium_df.loc[1, 'TEAM'] = 'CHICAGOBEARS'
stadium_df.set_index('TEAM', inplace=True)
stadium_df.rename(columns={'Geo Point': 'geo_point'}, inplace=True)
stadium_df

,NAME,geo_point,ROOF_TYPE,Station
TEAM,,,,
DALLASCOWBOYS,Cowboys Stadium,"32.746930527,-97.0923739136",Retractable,USW00053961
CHICAGOBEARS,Soldier Field,"41.8625000675,-87.6167699762",Open,USW00054811
NEWENGLANDPATRIOTS,Gillette Stadium,"42.0918799131,-71.2649100654",Open,USW00054796
CAROLINAPANTHERS,Bank of America Stadium,"35.2258400005,-80.8533099799",Open,USW00092821
TAMPABAYBUCCANEERS,Raymond James Stadium,"27.978840052,-82.5034900566",Open,USW00092827
SEATTLESEAHAWKS,Qwest Stadium,"47.5952976774,-122.331714406",Open,USW00004237
JACKSONVILLEJAGUARS,Jacksonville Municipal Stadium,"30.3238700522,-81.6368999561",Open,USW00063856
HOUSTONTEXANS,Reliant Stadium,"29.6849299406,-95.410919978",Retractable,USW00053960
PHILADELPHIAEAGLES,Lincoln Financial Field,"39.9014700517,-75.1672900567",Open,USW00003761


In [4]:
games_df = pd.read_csv('data_from_web/nfl_2018_games.csv')
games_df['Winner'] = games_df['Winner/tie']
games_df['Loser'] = games_df['Loser/tie']
hometeam = []
awayteam = []
for index,row in games_df.iterrows():
    if row[5]=='@':
        hometeam.append(row['Loser/tie'])
        awayteam.append(row['Winner/tie'])
    else:
        hometeam.append(row['Winner/tie'])
        awayteam.append(row['Loser/tie'])
games_df['Home Team'] = hometeam
games_df['Away Team'] = awayteam
games_df.drop(columns=['Day', 'Unnamed: 5', 'Unnamed: 7', 'Winner/tie', 'Loser/tie', 'TOW', 'TOL'], inplace = True)
games_df.reset_index(inplace=True)
games_df.rename(columns={'index':'Game ID'}, inplace=True)
games_df.set_index('Game ID', inplace=True)
date_obj = []
for index,row in games_df.iterrows():
    date_obj.append(dt.datetime.strptime(row['Date']+' 2018', '%B %d %Y'))
games_df['Datetime Obj'] = date_obj
stadium = []
station = []
for index,row in games_df.iterrows():
    team = row['Home Team'].upper().replace(' ', '')
    if team == 'NEWYORKGIANTS' or team == 'NEWYORKJETS':
        team = 'NEWYORKGIANTS/NEWYORKJETS'
    if team == 'LOSANGELESCHARGERS' or team =='LOSANGELESRAMS':
        team = 'LOSANGELESCHARGERS/LOSANGELESRAMS'
    stadium.append(stadium_df['NAME'].loc[team])
    station.append(stadium_df['Station'].loc[team])
games_df['Stadium'] = stadium
games_df['Station'] = station
games_df


,Week,Date,Time,PtsW,PtsL,YdsW,YdsL,Winner,Loser,Home Team,Away Team,Datetime Obj,Stadium,Station
Game ID,,,,,,,,,,,,,,
0,1,September 6,8:20PM,18,12,232,299,Philadelphia Eagles,Atlanta Falcons,Philadelphia Eagles,Atlanta Falcons,2018-09-06,Lincoln Financial Field,USW00003761
1,1,September 9,1:00PM,48,40,529,475,Tampa Bay Buccaneers,New Orleans Saints,New Orleans Saints,Tampa Bay Buccaneers,2018-09-09,Louisiana Superdome,USW00053960
2,1,September 9,1:00PM,24,16,343,327,Minnesota Vikings,San Francisco 49ers,Minnesota Vikings,San Francisco 49ers,2018-09-09,Hubert H. Humphrey Metrodome,USW00054854
3,1,September 9,1:00PM,27,20,342,336,Miami Dolphins,Tennessee Titans,Miami Dolphins,Tennessee Titans,2018-09-09,Sun Life Stadium,USW00092821
4,1,September 9,1:00PM,20,15,305,324,Jacksonville Jaguars,New York Giants,New York Giants,Jacksonville Jaguars,2018-09-09,Meadowlands Stadium,USW00064756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,17,December 30,4:25PM,26,24,463,426,Baltimore Ravens,Cleveland Browns,Baltimore Ravens,Cleveland Browns,2018-12-30,M&T Bank Stadium,USW00064758
252,17,December 30,4:25PM,35,3,409,292,Kansas City Chiefs,Oakland Raiders,Kansas City Chiefs,Oakland Raiders,2018-12-30,Arrowhead Stadium,USW00023908
253,17,December 30,4:25PM,24,0,360,89,Philadelphia Eagles,Washington Redskins,Washington Redskins,Philadelphia Eagles,2018-12-30,FedEx Field,USW00003761


In [54]:
for index, row in games_df.iterrows():
    base = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&units=standard&startdate='
    startDate = row['Datetime Obj'].strftime('%Y-%m-%d')
    endDate = startDate
    station=row['Station']
    url = f'{base}{startDate}&enddate={endDate}&stationid=GHCND:{station}'
    head = {"token": os.getenv('NOAA_TOK')}
    json = requests.get(url, headers=head).json()
    for result in json['results']:
        if result['datatype']=='PRCP':
            games_df.loc[index, 'Rain'] = result['value']
        elif result['datatype']=='TMAX':
            games_df.loc[index, 'Temp'] = result['value']
games_df.dropna(how='any', inplace=True) #There seems to be only one value returning NaN from the API, So i am dropping this record.
games_df.drop(columns=['Date', 'Time'], inplace=True)
games_df

,Week,Date,Time,PtsW,PtsL,YdsW,YdsL,Winner,Loser,Home Team,Away Team,Datetime Obj,Stadium,Station,Rain,Temp
Game ID,,,,,,,,,,,,,,,,
0,1,September 6,8:20PM,18,12,232,299,Philadelphia Eagles,Atlanta Falcons,Philadelphia Eagles,Atlanta Falcons,2018-09-06,Lincoln Financial Field,USW00003761,0.00,90.0
1,1,September 9,1:00PM,48,40,529,475,Tampa Bay Buccaneers,New Orleans Saints,New Orleans Saints,Tampa Bay Buccaneers,2018-09-09,Louisiana Superdome,USW00053960,0.12,89.0
2,1,September 9,1:00PM,24,16,343,327,Minnesota Vikings,San Francisco 49ers,Minnesota Vikings,San Francisco 49ers,2018-09-09,Hubert H. Humphrey Metrodome,USW00054854,0.04,64.0
3,1,September 9,1:00PM,27,20,342,336,Miami Dolphins,Tennessee Titans,Miami Dolphins,Tennessee Titans,2018-09-09,Sun Life Stadium,USW00092821,0.06,86.0
4,1,September 9,1:00PM,20,15,305,324,Jacksonville Jaguars,New York Giants,New York Giants,Jacksonville Jaguars,2018-09-09,Meadowlands Stadium,USW00064756,0.00,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,17,December 30,4:25PM,26,24,463,426,Baltimore Ravens,Cleveland Browns,Baltimore Ravens,Cleveland Browns,2018-12-30,M&T Bank Stadium,USW00064758,0.05,34.0
252,17,December 30,4:25PM,35,3,409,292,Kansas City Chiefs,Oakland Raiders,Kansas City Chiefs,Oakland Raiders,2018-12-30,Arrowhead Stadium,USW00023908,0.01,39.0
253,17,December 30,4:25PM,24,0,360,89,Philadelphia Eagles,Washington Redskins,Washington Redskins,Philadelphia Eagles,2018-12-30,FedEx Field,USW00003761,0.00,NaN


In [64]:
games_df.rename(columns={'Home Team': 'home_team',
'Away Team':'away_team',
'Datetime Obj': 'date'}, inplace=True)
games_df

,Week,PtsW,PtsL,YdsW,YdsL,Winner,Loser,home_team,away_team,date,Stadium,Station,Rain,Temp
Game ID,,,,,,,,,,,,,,
0,1,18,12,232,299,Philadelphia Eagles,Atlanta Falcons,Philadelphia Eagles,Atlanta Falcons,2018-09-06,Lincoln Financial Field,USW00003761,0.00,90.0
1,1,48,40,529,475,Tampa Bay Buccaneers,New Orleans Saints,New Orleans Saints,Tampa Bay Buccaneers,2018-09-09,Louisiana Superdome,USW00053960,0.12,89.0
2,1,24,16,343,327,Minnesota Vikings,San Francisco 49ers,Minnesota Vikings,San Francisco 49ers,2018-09-09,Hubert H. Humphrey Metrodome,USW00054854,0.04,64.0
3,1,27,20,342,336,Miami Dolphins,Tennessee Titans,Miami Dolphins,Tennessee Titans,2018-09-09,Sun Life Stadium,USW00092821,0.06,86.0
4,1,20,15,305,324,Jacksonville Jaguars,New York Giants,New York Giants,Jacksonville Jaguars,2018-09-09,Meadowlands Stadium,USW00064756,0.00,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,17,48,32,377,391,Los Angeles Rams,San Francisco 49ers,Los Angeles Rams,San Francisco 49ers,2018-12-30,Los Angeles Memorial Coliseum,USW00053151,0.00,63.0
251,17,26,24,463,426,Baltimore Ravens,Cleveland Browns,Baltimore Ravens,Cleveland Browns,2018-12-30,M&T Bank Stadium,USW00064758,0.05,34.0
252,17,35,3,409,292,Kansas City Chiefs,Oakland Raiders,Kansas City Chiefs,Oakland Raiders,2018-12-30,Arrowhead Stadium,USW00023908,0.01,39.0


In [63]:
teams_df = pd.read_csv('data_from_web/nfl_teams.csv')
teams_df = teams_df[['team_name', 'team_conference', 'team_division']]
teams_df

,team_name,team_conference,team_division
0,Arizona Cardinals,NFC,NFC West
1,Phoenix Cardinals,NFC,NaN
2,St. Louis Cardinals,NFC,NaN
3,Atlanta Falcons,NFC,NFC South
4,Baltimore Ravens,AFC,AFC North
5,Buffalo Bills,AFC,AFC East
6,Carolina Panthers,NFC,NFC South
7,Chicago Bears,NFC,NFC North
8,Cincinnati Bengals,AFC,AFC North
9,Cleveland Browns,AFC,AFC North


In [52]:
url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMAX,PRCP,WIPRCP&units=standard&startdate=2018-11-11&enddate=2018-11-11&stationid=GHCND:USW00054808'
requests.get(url, headers={"token": os.getenv('NOAA_TOK')}).json()

{'metadata': {'resultset': {'offset': 1, 'count': 1, 'limit': 25}},
 'results': [{'date': '2018-11-11T00:00:00',
   'datatype': 'TMAX',
   'station': 'GHCND:USW00054808',
   'attributes': ',,W,',
   'value': 43.0}]}